In [4]:
import numpy as np
import pandas as pd
import glob


In [5]:
# pd.set_option('display.max_rows', None)


In [6]:
def load_csv_to_df(files):
    df = pd.concat((pd.read_csv(f, skipinitialspace = True) for f in files), ignore_index=True)
    return df


In [7]:
tasks_joined_files = glob.glob(
    "../data/Update Travel Time Archive/Travel Time Task/TravelTimeTaskArchive*.csv")
df_tasks = load_csv_to_df(tasks_joined_files)

summary_joined_files = glob.glob(
    "../data/Update Travel Time Archive/Travel Time Summary/TravelTimeSummaryTaskArchive*.csv")
df_summary = load_csv_to_df(summary_joined_files)


,AssetNumber,AssetIdentifier,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,99945336683,SB HWY-5 Thompson-Nicola • 191634,12/18/2020 9:40:03 PM,215.7333,211.3526,4.3807,OK,340.139
1,99945336683,SB HWY-5 Thompson-Nicola • 191634,12/18/2020 9:45:03 PM,215.7333,211.3526,4.3807,OK,340.139
2,99945336683,SB HWY-5 Thompson-Nicola • 191634,12/18/2020 9:50:03 PM,215.3667,211.3526,4.0141,OK,340.139
3,99945336683,SB HWY-5 Thompson-Nicola • 191634,12/18/2020 9:55:03 PM,215.3667,211.3526,4.0141,OK,340.139
4,99945336683,SB HWY-5 Thompson-Nicola • 191634,12/18/2020 10:00:03 PM,215.4167,211.3526,4.0641,OK,340.139
...,...,...,...,...,...,...,...,...
13438804,23473284594,MRPP-AP 14 • NB • Ledcor Sicim LP • 422.8,7/9/2021 1:57:09 AM,0.0000,0.0000,0.0000,OK,3.160
13438805,23473284594,MRPP-AP 14 • NB • Ledcor Sicim LP • 422.8,7/9/2021 1:57:09 AM,0.0000,0.0000,0.0000,OK,3.160
13438806,23473284594,MRPP-AP 14 • NB • Ledcor Sicim LP • 422.8,7/9/2021 1:57:09 AM,0.0000,0.0000,0.0000,OK,3.160
13438807,23473284594,MRPP-AP 14 • NB • Ledcor Sicim LP • 422.8,7/9/2021 1:57:09 AM,0.0000,0.0000,0.0000,OK,3.160


,AssetNumber,AssetIdentifier,CreatedDate,SummaryDate,UnderConstructionAvgTime,BaselineAvgTime,DelayAvgTime
0,80002045894,SB HWY-5 Fort George • 191005,12/18/2020 1:00:00 AM,12/18/2020 12:00:00 AM,147.9389,150.3108,0.0000
1,18939973379,NB HWY-5 Thompson-Nicola • 191007,12/18/2020 1:00:00 AM,12/18/2020 12:00:00 AM,222.4253,213.7520,8.8015
2,99945336683,SB HWY-5 Thompson-Nicola • 191634,12/18/2020 1:00:00 AM,12/18/2020 12:00:00 AM,214.6708,211.3526,3.5020
3,48672805900,Complete Route NB • 191644,12/18/2020 1:00:00 AM,12/18/2020 12:00:00 AM,387.7015,379.7691,8.0261
4,47044236603,SB - Lower Mainland • 191636,12/18/2020 1:00:00 AM,12/18/2020 12:00:00 AM,17.6833,17.2886,0.3947
...,...,...,...,...,...,...,...
145260,35374012671,Spread 4B - SB,6/20/2021 1:00:00 AM,6/20/2021 12:00:00 AM,43.5600,45.6200,0.0600
145261,76957583556,Spread 3/4A - SB,6/20/2021 1:00:00 AM,6/20/2021 12:00:00 AM,124.4100,128.6200,0.0300
145262,59948387105,Spread 3/4A - NB,6/20/2021 1:00:00 AM,6/20/2021 12:00:00 AM,123.3600,127.3800,0.0100
145263,33379829945,MRPP-AP 2 • SB • Ledcor Sicim LP • 408.8,6/20/2021 1:00:00 AM,6/20/2021 12:00:00 AM,1.8600,1.8700,0.0000


In [9]:
df_asset = pd.concat([df_tasks[['AssetNumber', 'AssetIdentifier']], df_summary[['AssetNumber', 'AssetIdentifier']]], axis=0) 
df_unique = df_asset.drop_duplicates()
# df_unique.to_csv('../data/unique_assets.csv', index = False)
df_unique


,AssetNumber,AssetIdentifier
0,99945336683,SB HWY-5 Thompson-Nicola • 191634
1,99945336683,SB HWY-5 Thompson-Nicola • 191634
2,99945336683,SB HWY-5 Thompson-Nicola • 191634
3,99945336683,SB HWY-5 Thompson-Nicola • 191634
4,99945336683,SB HWY-5 Thompson-Nicola • 191634
...,...,...
145260,35374012671,Spread 4B - SB
145261,76957583556,Spread 3/4A - SB
145262,59948387105,Spread 3/4A - NB
145263,33379829945,MRPP-AP 2 • SB • Ledcor Sicim LP • 408.8


,AssetNumber,AssetIdentifier
0,99945336683,SB HWY-5 Thompson-Nicola • 191634
8,48672805900,Complete Route NB • 191644
9,18939973379,NB HWY-5 Thompson-Nicola • 191007
10,96802706037,Complete Route SB • 191655
111,36674989230,NB - Lower Mainland • 191637
...,...,...
9868974,54084450653,AP 4-B-133-A • NB • Ledcor Sicim LP • 752.7
9906636,17682621065,"AP 3-C-75-A, AP 3-C-76-B • SB • Ledcor Sicim L..."
10552425,96861493271,AP 5-B-107 • N/A • Surerus Murphy JV • 811.1
10552544,18661378775,MRPP-AP 9-C • NB • Ledcor Sicim LP • 419.2


In [76]:
df_filtered = df_merged[(df_merged['UnderConstructionTime'] != 0) & (df_merged['BaselineTime'] != 0)]
df_filtered


,AssetNumber,CreatedDate,UnderConstructionTime,BaselineTime,DelayTime,State,SegmentLength
0,99945336683,12/18/2020 9:40:03 PM,215.7333,211.3526,4.3807,OK,340.139
1,99945336683,12/18/2020 9:45:03 PM,215.7333,211.3526,4.3807,OK,340.139
2,99945336683,12/18/2020 9:50:03 PM,215.3667,211.3526,4.0141,OK,340.139
3,99945336683,12/18/2020 9:55:03 PM,215.3667,211.3526,4.0141,OK,340.139
4,99945336683,12/18/2020 10:00:03 PM,215.4167,211.3526,4.0641,OK,340.139
...,...,...,...,...,...,...,...
9438803,19813765728,6/20/2021 9:20:13 AM,2.6500,3.4500,0.0000,OK,5.300
9438805,61149462439,6/20/2021 9:20:13 AM,4.1700,4.5300,0.0000,OK,7.920
9438806,89050175791,6/20/2021 9:20:13 AM,2.8800,3.5300,0.0000,OK,5.290
9438807,83015587391,6/20/2021 9:20:13 AM,3.9300,4.5200,0.0000,OK,7.900


In [81]:
query = 80002045894


In [ ]:
df_filtered.loc[df_filtered['AssetNumber'] == query]
